## Content

* [Identifying the technology used by a website](#Identifying-the-technology-used-by-a-website)
* [Finding the owner of a website](#Finding-the-owner-of-a-website)
* [Crawl a website](#crawl-a-website)
* [Scraping the data](#Scraping-the-data)

#### Identifying the technology used by a website

In [4]:
import builtwith
builtwith.parse('https://mypage.i-exam.ru/')

{'web-servers': ['Nginx'],
 'web-frameworks': ['Twitter Bootstrap'],
 'javascript-frameworks': ['jQuery']}

#### Finding the owner of a website

In [5]:
import whois
whois.whois('https://mypage.i-exam.ru/')

{'domain_name': 'I-EXAM.RU',
 'registrar': 'RU-CENTER-RU',
 'creation_date': datetime.datetime(2008, 4, 21, 20, 0),
 'expiration_date': datetime.datetime(2022, 4, 21, 21, 0),
 'name_servers': ['ns3-l2.nic.ru.',
  'ns4-cloud.nic.ru.',
  'ns4-l2.nic.ru.',
  'ns8-cloud.nic.ru.',
  'ns8-l2.nic.ru.'],
 'status': 'REGISTERED, DELEGATED, VERIFIED',
 'emails': None,
 'org': '"Institute of Quality Monitoring Ltd"'}

### Crawl a website

In order to scrape a website, we first need to download its web pages containing the
data of interest—a process known as crawling. 
Three common approaches to crawling
a website:
* Crawling a sitemap
* Iterating the database IDs of each web page
* Following web page links

In [38]:
# download a web page
import urllib
def download(url, user_agent='wswp', num_retries=2):
    '''
        url - 
        user_agent - preferable to use an identifiable
                     user agent in case problems occur
                     with our web crawler. Also, some
                     websites block this default user
                     agent, perhaps after they experienced
                     a poorly made Python web crawler
                     overloading their server.
                     
        num_retries - 
        
    '''
    print('Downloading:', url)
    
    headers = {'User-agent': user_agent}
    request = urllib.request.Request(url, headers=headers)
    
    try:
        html = urllib.request.urlopen(request).read().decode('utf-8')
    
    except urllib.error.URLError as e:
        print(f'Download error: {e}')
        html = None
        
        if num_retries > 0:
            if hasattr(e, 'code') and 500 <= e.code < 600:
                return download(url, num_retries - 1)
    
    return html

In [39]:
import re
def crawl_sitemap(url):
    # download the sitemap file
    sitemap = download(url)
    # extract the sitemap links
    links = re.findall('<loc>(.*?)</loc>', sitemap) # sitemap parse
    # download each link
    for link in links:
        html = download(link)
        # scrape html here
        # ...
        
        
# ID iteration crawler
import itertools
for page in itertools.count(1):
    url = 'http://example.webscraping.com/view/-%d' % page
    html = download(url)
    if html is None:
        break
    else:
        # success - can scrape the result
        pass
    
# Link crawler
import urlparse
def link_crawler(seed_url, link_regex):
    crawl_queue = [seed_url]
    # keep track which URL's have seen before
    seen = set(crawl_queue)
    
    while crawl_queue:
        url = crawl_queue.pop()
        html = download(url)
        for link in get_links(html):
            # check if link matches expected regex
            if re.match(link_regex, link):
                # form absolute link
                link = urlparse.urljoin(seed_url, link)
                # check if have already seen this link
                if link not in seen:
                    seen.add(link)
                    crawl_queue.append(link)


def get_links(html):
    """
        Return a list of links from html
    """
    # a regular expression to extract all links from the webpage
    webpage_regex = re.compile('<a[^>]+href=["\'](.*?)["\']', re.IGNORECASE)
    # list of all links from the webpage
    return webpage_regex.findall(html)

In [40]:
http = crawl_sitemap('https://google.com')

Downloading: https://google.com


**Throttling downloads**.


If we crawl a website too fast, we risk being blocked or overloading the server.
To minimize these risks, we can throttle our crawl by waiting for a delay between
downloads. 


**Avoiding spider traps**.


Some websites dynamically generate their content and can have an infinite number
of web pages. For example, if the website has an online calendar with links provided
for the next month and year, then the next month will also have links to the next
month, and so on for eternity. This situation is known as a spider trap.
A simple way to avoid getting stuck in a spider trap is to track how many links
have been followed to reach the current web page, which we will refer to as depth.
Then, when a maximum depth is reached, the crawler does not add links from this
web page to the queue.
```
def link_crawler(..., max_depth=2):
    max_depth = 2
    seen = {}
    # ...
    depth = seen[url]
    if depth != max_depth:
        for link in links:
            if link not in seen:
                seen[link] = depth + 1
                crawl_queue.append(link)
```

### Scraping the data

We need to make this crawler achieve
something by extracting data from each web page, which is known as scraping.
There are three approaches to scrape a web page:

* regular expressions
* beautiful soup
* Lxml

Beautiful Soup is over six times slower than the
other two approaches when used to scrape our example web page. This result could
be anticipated because lxml and the regular expression module were written in C,
while BeautifulSoup is pure Python. An interesting fact is that lxml performed
comparatively well with regular expressions, since lxml has the additional overhead
of having to parse the input into its internal format before searching for elements. 


If the bottleneck to your scraper is downloading web pages rather than extracting
data, it would not be a problem to use a slower approach, such as Beautiful Soup.
Or, if you just need to scrape a small amount of data and want to avoid additional
dependencies, regular expressions might be an appropriate choice. However, in
general, lxml is the best choice for scraping, because it is fast and robust, while
regular expressions and Beautiful Soup are only useful in certain niches.

In [ ]:
# regular expressions
import re
url = 'http://example.webscraping.com/view/UnitedKingdom-239'
html = download(url)
re.findall('<td class="w2p_fw">(.*?)</td>', html)
re.findall('<td class="w2p_fw">(.*?)</td>', html)[1]
re.findall('<tr id="places_area__row"><td class="w2p_fl"><label for="places_area" \
            id="places_area__label">Area: </label></td><td class="w2p_fw">(.*?)</td>', html)

In [ ]:
# beautiful soup
from bs4 import BeautifulSoup
broken_html = '<ul class=country><li>Area<li>Population</ul>'
# parse the HTML
soup = BeautifulSoup(broken_html, 'html.parser')
fixed_html = soup.prettify()
print(fixed_html)
'''
    <html>
         <body>
             <ul class="country">
                 <li>Area</li>
                 <li>Population</li>
             </ul>
         </body>
    </html>
'''

ul = soup.find('ul', attrs={'class':'country'})
ul.find('li') # returns just the first match
# <li>Area</li>
ul.find_all('li')
# [<li>Area</li>, <li>Population</li>]

In [ ]:
# Lxml

import lxml.html
broken_html = '<ul class=country><li>Area<li>Population</ul>'
tree = lxml.html.fromstring(broken_html) # parse the HTML
fixed_html = lxml.html.tostring(tree, pretty_print=True)

'''
    we will use CSS selectors here and in future
    examples, because they are more compact 
    and can be reused later in when parsing dynamic conten
'''

tree = lxml.html.fromstring(html)
# This line finds a table row element
# with the places_area__row ID, and then selects the child table data tag with the
# w2p_fw class.
td = tree.cssselect('tr#places_area__row > td.w2p_fw')[0]
area = td.text_content()

**CSS Selectors.**

* Select any tag: *
* Select by tag <a></a>: a
* Select by class of "link": .link
* Select by tag <a></a> with class "link": a.link
* Select by tag <a></a> with ID "home": a#home
* Select by child <span> of tag <a></a>: a > span
* Select by descendant <span> of tag <a></a>: a span
* Select by tag <a></a> with attribute title of "Home": a[title=Home]

### Caching Downloads

In [ ]:
# Adding cache support to the link crawler
class Downloader:
    def __init__(self, delay=5,
        user_agent='wswp', proxies=None,
        num_retries=1, cache=None):
        self.throttle = Throttle(delay)
        self.user_agent = user_agent
        self.proxies = proxies
        self.num_retries = num_retries
        self.cache = cache
    
    def __call__(self, url):
        result = None
        
        if self.cache:
            try:
                result = self.cache[url]
            except KeyError:
                # url is not available in cache
                pass
            else:
                if self.num_retries > 0 and \
                500 <= result['code'] < 600:
                # server error so ignore result from cache
                # and re-download
                result = None
                
        if result is None:
            # result was not loaded from cache
            # so still need to download
            self.throttle.wait(url)
            proxy = random.choice(self.proxies) if self.proxies else None
            headers = {'User-agent': self.user_agent}
            
            result = self.download(url, headers, proxy, self.num_retries)
            if self.cache:
                # save result to cache
                self.cache[url] = result
        
        return result['html']
    
    def download(self, url, headers, proxy, num_retries, data=None):
        ...
        return {'html': html, 'code': code}


'''
    The link crawler also needs to be slightly updated to support caching by adding the
    cache parameter, removing the throttle, and replacing the download function with
    the new class
'''
    
def link_crawler(..., cache=None):
    crawl_queue = [seed_url]
    seen = {seed_url: 0}
    num_urls = 0
    rp = get_robots(seed_url)
    D = Downloader(delay=delay, user_agent=user_agent, proxies=proxies, num_retries=num_retries, cache=cache)
    
    while crawl_queue:
        url = crawl_queue.pop()
        depth = seen[url]
        # check url passes robots.txt restrictions
        if rp.can_fetch(user_agent, url):
            html = D(url)
            links = []
            ...

The interesting part of the Download class used in the preceding code is in the
__call__ special method, where the cache is checked before downloading. This
method first checks whether the cache is defined. If so, it checks whether this
URL was previously cached. If it is cached, it checks whether a server error was
encountered in the previous download. Finally, if no server error was encountered,
the cached result can be used. If any of these checks fail, the URL needs to be
downloaded as usual, and the result will be added to the cache. The download
method of this class is the same as the previous download function, except now it
returns the HTTP status code along with the downloaded HTML so that error codes
can be stored in the cache.

**Disk cache**

To keep our file path safe across filesystems, it needs to be restricted to
numbers, letters, basic punctuation, and replace all other characters with an
underscore, as shown in the following code:

```
>>> import re
>>> url = 'http://example.webscraping.com/default/view/
 Australia-1'
>>> re.sub('[^/0-9a-zA-Z\-.,;_ ]', '_', url)
'http_//example.webscraping.com/default/view/Australia-1'

```

Additionally, the filename and the parent directories need to be restricted to 255
characters (as shown in the following code) to meet the length limitations described
in the preceding table:

```
>>> filename = '/'.join(segment[:255] for segment in
 filename.split('/'))
```

In [ ]:
import os
import re
import urlparse
class DiskCache:
    def __init__(self, cache_dir='cache'):
        self.cache_dir = cache_dir
        self.max_length = max_length
        
    def url_to_path(self, url):
        """Create file system path for this URL
        """
        components = urlparse.urlsplit(url)
        # append index.html to empty paths
        path = components.path
        if not path:
            path = '/index.html'
        elif path.endswith('/'):
            path += 'index.html'
        filename = components.netloc + path + components.query
        # replace invalid characters
        filename = re.sub('[^/0-9a-zA-Z\-.,;_ ]', '_', filename)
        # restrict maximum number of characters
        filename = '/'.join(segment[:250] for segment in filename.split('/'))
        
        return os.path.join(self.cache_dir, filename)
    
    def __getitem__(self, url):
        """Load data from disk for this URL
        """
        path = self.url_to_path(url)
        if os.path.exists(path):
            with open(path, 'rb') as fp:
                return pickle.load(fp)
        else:
            # URL has not yet been cached
            raise KeyError(url + ' does not exist')
        
    def __setitem__(self, url, result):
        """Save data to disk for this url
        """
        path = self.url_to_path(url)
        folder = os.path.dirname(path)
        if not os.path.exists(folder):
            os.makedirs(folder)
        with open(path, 'wb') as fp:
            fp.write(pickle.dumps(result))

**Database cache**

To avoid the anticipated limitations to our disk-based cache, we will now build
our cache on top of an existing database system. When crawling, we may need to
cache massive amounts of data and will not need any complex joins, so we will use
a NoSQL database, which is easier to scale than a traditional relational database.
Specifically, our cache will use MongoDB, which is currently the most popular
NoSQL database.


NoSQL stands for Not Only SQL and is a relatively new approach to database
design. The traditional relational model used a fixed schema and splits the data into
tables. However, with large datasets, the data is too big for a single server and needs
to be scaled across multiple servers. This does not fit well with the relational model
because, when querying multiple tables, the data will not necessarily be available on
the same server. NoSQL databases, on the other hand, are generally schemaless and
designed from the start to shard seamlessly across servers. There have been multiple
approaches to achieve this that fit under the NoSQL umbrella. There are column data
stores, such as HBase; key-value stores, such as Redis; document-oriented databases,
such as MongoDB; and graph databases, such as Neo4j.

In [2]:
from datetime import datetime, timedelta
from pymongo import MongoClient
class MongoCache:
    def __init__(self, client=None, expires=timedelta(days=30)):
        # if a client object is not passed then try
        # connecting to mongodb at the default localhost port
        self.client = MongoClient('localhost', 27017) if client is None else client
        # create collection to store cached webpages,
        # which is the equivalent of a table
        # in a relational database
        self.db = client.cache
        # create index to expire cached webpages
        self.db.webpage.create_index('timestamp',
            expireAfterSeconds=expires.total_seconds())
    
    
    def __getitem__(self, url):
        """Load value at this URL
        """
        record = self.db.webpage.find_one({'_id': url})
        if record:
            return record['result']
        else:
            raise KeyError(url + ' does not exist')
            
    def __setitem__(self, url, result):
        """Save value for this URL
        """
        record = {'result': result, 'timestamp':
            datetime.utcnow()}
        self.db.webpage.update({'_id': url}, {'$set': record},
            upsert=True)